# PS4: Simulated Annealing for Function Optimization
In this activity, you will implement the Simulated Annealing algorithm to solve a classic problem in quantitative finance: the minimum variance portfolio allocation problem.

> __Learning Objectives:__
> 
> After completing this activity, students will be able to: 
> * **Formulate constrained portfolio optimization problems:** We develop the minimum variance portfolio allocation problem as a constrained quadratic programming problem that balances portfolio risk (variance) against target return requirements while ensuring non-negative weights that sum to one.
> * **Implement simulated annealing with penalty methods:** We use the simulated annealing algorithm with barrier and penalty terms to handle both inequality constraints (non-negativity) and equality constraints (target return and sum-to-one) in the augmented objective function.
> * **Evaluate optimization performance through comparison:** We train the simulated annealing model on S&P 500 historical data and compare the resulting portfolio allocations and risk metrics against solutions from traditional quadratic programming methods to assess convergence and solution quality.

So what do we need to do?

> __To Do:__
>
> In this problem set you will need to implement the Simulated Annealing algorithm in the `src/Compute.jl` file in the `solve` function. You will also need to tune the hyper-parameters and your implementation of simulated annealing algorithm. There is nothing to do in `Task 1`, as that is just data gathering and processing. You will need to implement the simulated annealing algorithm in `Task 2` and tune your implementation in `Task 3`.

This is a fun application of the Simulated Annealing algorithm to a real-world problem in finance. So, let's get started!

___

## Background: Minimum Variance Portfolio Allocation Problem
The goal of minimum variance portfolio allocation is to find the optimal weights $\mathbf{w}$ that minimize the portfolio risk for a given level of expected return, or equivalently, maximize the expected return for a given level of risk. This is typically done by solving a __constrained quadratic programming__ problem. 

### Portfolio Reward
The reward of a portfolio is measured by its expected growth (return), which is the weighted average of the expected growth rates (returns) of the individual assets in the portfolio. 

Suppose we have a portfolio $\mathcal{P}$ consisting of $M$ assets. Let $w_i\in\mathbb{R}_{\geq 0}$ be the weight of asset $i$ in the portfolio (i.e., the dollar fraction of the total portfolio value invested in asset $i$), and let $\mathbb{E}[g_i]$ be the expected growth rate (return) of asset $i$. Then, the expected growth rate (return) of the portfolio, denoted as $\mathbb{E}[g_{\mathcal{P}}]$, is given by:
$$
\mathbb{E}[g_{\mathcal{P}}] = \sum_{i=1}^{M} w_i \mathbb{E}[g_i]\quad\Longrightarrow\;\mathbf{w}^{\top}\mathbb{E}[\mathbf{g}]
$$
where $M$ is the total number of assets in the portfolio, i.e., $|\mathcal{P}| = M$, the weight vector is $\mathbf{w}^{\top} = [w_1, w_2, \dots, w_M]$, the sum of weights is one, and $\mathbb{E}[\mathbf{g}] = [\mathbb{E}[g_1], \mathbb{E}[g_2], \dots, \mathbb{E}[g_M]]^{\top}$ is the vector of expected growth rates (returns) of the individual assets.

### Portfolio Risk
The risk of a portfolio is (typically) measured by its variance (or standard deviation) of growth rates (returns), which takes into account the variances of the individual assets as well as the covariances between them. The portfolio variance written in terms of growth rates is given by:
$$
\text{Var}(g_{\mathcal{P}}) = \sum_{i=1}^{M} \sum_{j=1}^{M} w_i w_j \text{Cov}(g_i, g_j)\quad\Longrightarrow\;\mathbf{w}^{\top}\mathbf{\Sigma}_{g}\mathbf{w}
$$
where $\text{Cov}(g_i, g_j)$ is the covariance between the growth rates of assets $i$ and $j$, and $\mathbf{\Sigma}_{g}$ is the covariance matrix of asset growth rates, defined as:
$$
\mathbf{\Sigma}_{g} =
\begin{bmatrix}
\text{Var}(g_1) & \text{Cov}(g_1, g_2) & \cdots & \text{Cov}(g_1, g_M) \\
\text{Cov}(g_2, g_1) & \text{Var}(g_2) & \cdots & \text{Cov}(g_2, g_M) \\
\vdots & \vdots & \ddots & \vdots \\
\text{Cov}(g_M, g_1) & \text{Cov}(g_M, g_2) & \cdots & \text{Var}(g_M)
\end{bmatrix}
$$

### Optimization Problem Formulation
Let's consider the case when we have a portfolio $\mathcal{P}$ consisting of $M$ __risky assets__, i.e., only equity, ETFs (or potentially derivatives) but no fixed income assets. In this case, we can formulate the optimal weights optimization problem (written in terms of growth rate) as:

$$
\boxed{
\begin{align*}
\text{minimize}~\text{Var}(g_{\mathcal{P}}) &= \sum_{i\in\mathcal{P}}\sum_{j\in\mathcal{P}}w_{i}w_{j}\underbrace{\text{Cov}\left(g_{i},g_{j}\right)}_{= \sigma_{i}\sigma_{j}\rho_{ij}}\quad{\Longleftrightarrow\mathbf{w}^\top \mathbf{\Sigma}_{g} \mathbf{w}} \\
\text{subject to}~\mathbb{E}(g_{\mathcal{P}})& =  \sum_{i\in\mathcal{P}}w_{i}\;\mathbb{E}(g_{i})= R^{*}\quad\Longleftrightarrow\mathbf{w}^\top \mathbb{E}(\mathbf{g}) = R^{*} \\
\sum_{i\in\mathcal{P}}w_{i} & =  1 \\
w_{i} & \geq  0\quad\forall{i}\in\mathcal{P}
\end{align*}}
$$
The term $R^{*}$ is the target annualized growth rate (return) for portfolio $\mathcal{P}$ specified by the investor. The $w_{i}\geq{0}~\forall{i}\in\mathcal{P}$ and the summation-to-unity constraints forbid short selling (borrowing). If short selling (borrowing) is allowed, these constraints can be relaxed.

This is a __constrained quadratic programming__ problem, which can be solved using various optimization techniques, including __Simulated Annealing__.

___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include-student.jl")); # include the Include.jl file

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Data
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P 500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2014` until `12-31-2024`, along with data for a few exchange-traded funds and volatility products during that time. 

Let's load the `original_dataset::DataFrame` by calling [the `MyTrainingMarketDataSet()` function](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/data/#VLDataScienceMachineLearningPackage.MyTrainingMarketDataSet) and remove firms that do not have the maximum number of trading days. The cleaned dataset $\mathcal{D}$ will be stored in the `dataset` variable.

In [2]:
original_dataset = MyTrainingMarketDataSet() |> x-> x["dataset"];

Not all tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquisition or de-listing events. Let's collect only those tickers with the maximum number of trading days.

First, let's compute the number of records for a firm that we know has a maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_days::Int64` variable:

In [3]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow # nrow? (check out: DataFrames.jl)

2767

Now, let's iterate through our data and collect only tickers with `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [4]:
dataset = let

    # initialize -
    dataset = Dict{String, DataFrame}();

    # iterate through the dictionary; we can't guarantee a particular order
    for (ticker, data) ∈ original_dataset  # we get each (K, V) pair!
        if (nrow(data) == maximum_number_trading_days) # what is this doing?
            dataset[ticker] = data;
        end
    end
    dataset; # return
end;

Finally, let's get a list of the firms in our cleaned-up dataset (and sort them alphabetically). We store the sorted firm ticker symbols in the `list_of_tickers::Array{String,1}` variable.

In [5]:
list_of_tickers = keys(dataset) |> collect |> sort; # list of firm "ticker" symbols in alphabetical order

### Constants
Let's define a few constants that we will use throughout this notebook:

In [6]:
desired_target_return = 0.15; # desired target return for the portfolio

___

## Task 1: Compute the growth rate matrix
In this task, we compute the growth rate array which contains, for each day and each firm in our dataset, the value of the growth rate between time $j$ and $j-1$. 

>  __Continuously Compounded Growth Rate (CCGR)__
>
> Let's assume a model of the share price of firm $i$ is governed by an expression of the form:
>$$
\begin{align*}
S^{(i)}_{j} &= S^{(i)}_{j-1}\;\exp\left(g^{(i)}_{j,j-1}\Delta{t}_{j}\right)
\end{align*}
> $$
> where $S^{(i)}_{j-1}$ denotes the share price of firm $i$ at time index $j-1$, $S^{(i)}_{j}$ denotes the share price of firm $i$ at time index $j$, and $\Delta{t}_{j} = t_{j} - t_{j-1}$ denotes the length of a time step (units: years) between time index $j-1$ and $j$. The value we are going to estimate is the growth rate $g^{(i)}_{j,j-1}$ (units: inverse years) for each firm $i$ and each time step in the dataset.

We've implemented [the `log_growth_matrix(...)` function](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/data/#VLDataScienceMachineLearningPackage.log_growth_matrix) which takes the cleaned-up dataset and a list of ticker symbols, and returns the growth rate array. Each row of the growth rate array is a time step, while each column corresponds to a firm from the `list_of_tickers::Array{String,1}` array.

In [7]:
growth_rate_array = let

    # initialize -
    Δt = (1/252); # time-step one-day in units of years (trading year is 252 days)
    r̄ = 0.0; # assume the risk-free rate is 0

    # compute the growth matrix -
    growth_rate_array = log_growth_matrix(dataset, list_of_tickers, Δt = Δt, 
        risk_free_rate = r̄); # other optional parameters are at their defaults

    growth_rate_array; # return
end

2766×424 Matrix{Float64}:
 -0.877554    6.28105    -2.87097     …  -0.755391   0.245894  -1.00527
  2.81626     1.07149     1.39239         2.13832   -0.80279    0.986468
  3.31305     0.855597    0.00536803      0.109877   1.191     -2.58144
  0.646425   17.2599      1.69215         0.274716   3.1593    -0.368228
  1.81609     2.57961     3.31924         0.621677  -2.1687     4.40309
  0.61383    -3.96384    -0.79278     …  -0.862739  -1.90977   -3.11624
  2.86071    -0.483751    4.84573         1.7657    -1.77685   -1.0896
  2.04671     1.0135      1.90809         1.67597    4.44984   -0.137819
  1.31289     1.67413     0.107259       -1.50708   -2.13696    1.43784
  1.22016     6.12957     0.932578       -1.53202    2.87784   -1.43626
  ⋮                                   ⋱                        
 -4.36889     3.84443    -2.37452        -4.26011   -9.17906   -3.94641
 -2.51182    -2.60891   -10.1209         -3.03895   -7.07468   -7.14019
  2.21355     4.15066     7.27678         3.

> **Growth Rate Matrix Structure**
>
> The `growth_rate_array` is a matrix $\mathbf{G} \in \mathbb{R}^{m \times n}$ where each **row** represents a trading day (time step) in our dataset, each **column** represents a firm from the S&P 500, and each **element** $G_{i,j}$ contains the continuously compounded growth rate for firm $j$ on day $i$.

The matrix has 424 firms (columns) and $T-1$ = 2,766 trading days (rows), capturing the daily growth rate dynamics of the S&P 500 components from 2014 to 2024. 

Let's approximate the expected growth rate vector $\mathbb{E}[\mathbf{g}]$ and the covariance matrix $\mathbf{\Sigma}_{g}$ using the `growth_rate_array`.

The expected growth rate vector $\mathbb{E}[\mathbf{g}]$ can be approximated by computing the mean of each column in the `growth_rate_array`. Let's save the expected growth rate vector in the `ḡ::Array{Float64,1}` variable:

In [8]:
ḡ = mean(growth_rate_array, dims = 1) |> vec # expected growth rate vector (units: inverse years)

424-element Vector{Float64}:
  0.10874477980244975
 -0.03703243517086077
 -0.0799094577252642
  0.23280114847444194
  0.11114647191084896
  0.09794979247859234
  0.1333019961724274
  0.1835705859232435
  0.1326786226774918
  0.01415870983447601
  ⋮
 -0.0751151192174708
  0.08180190853490005
  0.006782878039138938
 -0.08549763123281434
  0.11052104541338774
  0.052025477386285615
  0.1796939974365368
  0.05468756164364767
  0.14764000690210635

The covariance matrix $\mathbf{\Sigma}_{g}$ can be approximated by computing the sample covariance of the `growth_rate_array` using [the `cov(...)` function](https://docs.julialang.org/en/v1/stdlib/Statistics/#Statistics.cov). We scale the covariance matrix by $\Delta t$ to convert from daily to annualized units. 

Let's save the covariance matrix in the `Σ̂::Array{Float64,2}` variable:

In [9]:
Σ̂ = let
    Δt = (1/252); # time-step one-day in units of years (trading year is 252 days)
    R = growth_rate_array; # shorthand
    Σ = cov(R, dims=1)*(Δt); # covariance matrix (units: dimensionless)
end

424×424 Matrix{Float64}:
 0.0535771   0.032397    0.0212982  …  0.0355662   0.0280938   0.0266666
 0.032397    0.207093    0.0431437     0.051429    0.0689041   0.0280481
 0.0212982   0.0431437   0.117391      0.0308539   0.037867    0.019717
 0.0229702   0.0312627   0.0163174     0.032633    0.0201827   0.022609
 0.0179206   0.0176175   0.0158407     0.0157326   0.0168026   0.0185472
 0.0244078   0.0195811   0.0145372  …  0.023159    0.0167416   0.0221966
 0.025643    0.0334402   0.0218931     0.0313218   0.0280694   0.023824
 0.0294242   0.02952     0.0185233     0.0369478   0.0198333   0.0262802
 0.0297618   0.0458278   0.0231343     0.0433011   0.0335821   0.0233085
 0.0169025   0.0325409   0.0206063     0.0219846   0.0327002   0.0131991
 ⋮                                  ⋱                          
 0.0339199   0.0921199   0.035255   …  0.0503196   0.0581754   0.0308858
 0.00888225  0.00814595  0.0118987     0.00616705  0.00592554  0.0112662
 0.0161836   0.0376862   0.0190661    

___

## Task 2: Solve the optimal allocation problem using Simulated Annealing
In this task, we will implement the Simulated Annealing algorithm to solve the minimum variance portfolio allocation problem formulated above. 

> __In practice:__ While the portfolio optimization problem can be solved efficiently using quadratic programming, simulated annealing offers a flexible alternative that can handle non-convex objectives and additional constraints without requiring derivatives. This is an example of a constrained optimization problem, which we can solve using Simulated Annealing by incorporating the constraints into the objective function using penalty terms.

In the general case, we can write the (augmented) objective function $P_{\mu,\rho}(x)$:
$$
\begin{align*}
    \min_{x\in\mathbb{R}^n}\;P_{\mu,\rho}(x)\;&=f(x)\;-\;\frac{1}{\mu}\sum_{i=1}^m\ln\bigl(-\,g_i(x)\bigr)\;+\;\frac{1}{2\rho}\sum_{j=1}^p 
    \bigl[h_j(x)\bigr]^2,\quad\text{where}\quad\mu>0,\;\rho>0\\
\end{align*}
$$
where $f(x)$ is the objective function, $g_i(x)$ are the inequality constraints, and $h_j(x)$ are the equality constraints. For our allocation problem, we have both equality and inequality constraints. The equality constraints are $\mathbf{w}^\top \mathbb{E}(\mathbf{g}) = R^{*}$ and $\sum_{i\in\mathcal{P}}w_{i} = 1$, while the inequality constraints are $w_{i} \geq 0~\forall{i}\in\mathcal{P}$. 

Putting this all together, we can write the augmented objective function for our allocation problem as:
$$
\boxed{
\begin{align*}
    \min_{\mathbf{w}\in\mathbb{R}^M}\;P_{\mu,\rho}(\mathbf{w})\;&=\;\mathbf{w}^{\top}\mathbf{\Sigma}_{g}\mathbf{w}\;-\;\frac{1}{\mu}\sum_{i=1}^M\ln\bigl(w_i\bigr)\;+\;\frac{1}{2\rho}\Bigr[\bigl(\mathbf{w}^\top ḡ - R^{*}\bigr)^2\;+\;\bigl(1-\sum_{i=1}^M w_i\bigr)^2\Bigl]\quad\blacksquare\\
\end{align*}}
$$
where $\mu>0$ and $\rho>0$ are penalty parameters that control the strength of the penalty terms. In our case, we will _decrease_ the value of $\mu$ and $\rho$ as the algorithm progresses, which will increase the strength of the penalty terms (as $\frac{1}{\mu}$ and $\frac{1}{\rho}$ grow larger) and force the solution to satisfy the constraints more closely.

### Implementation
Let's select a subset of firms from the S&P 500 to construct our portfolio. We'll choose a subset of firms and store their ticker symbols in the `my_list_of_tickers::Array{String,1}` array:

In [10]:
my_list_of_tickers = ["AAPL", "MSFT", "INTC", "MU", "AMD", "GS", "BAC", "WFC", "C", "F", "GM", 
    "JNJ", "PG", "UPS", "COST", "TGT", "WMT", "MRK", "PFE", "ADBE"]; # tickers selected for portfolio

Now that we have selected the firms in our risky portfolio, we'll compute the firm-specific expected return vector and covariance matrix. We'll store these in the $\bar{g}_{p}$ and $\hat{\Sigma}_{p}$ variables. 

In [11]:
ḡₚ, Σ̂ₚ = let

    # initialize -
    M = length(my_list_of_tickers); # number of assets in portfolio
    ḡₚ = Array{Float64,1}(); # drift vector
    Σ̂ₚ = Array{Float64,2}(undef, M, M); # covariance matrix for *our* portfolio

    # compute drift vector -
    for ticker ∈ my_list_of_tickers
        i = findfirst(x-> x == ticker, list_of_tickers); # find index of ticker in list_of_tickers
        push!(ḡₚ, ḡ[i]); # append drift value to ḡₚ
    end

    # compute covariance matrix -
    for i ∈ 1:M
        for j ∈ 1:M
            row_index = findfirst(x-> x == my_list_of_tickers[i], list_of_tickers); # find row index in full covariance matrix
            col_index = findfirst(x-> x == my_list_of_tickers[j], list_of_tickers); # find column index in full covariance matrix
            Σ̂ₚ[i,j] = Σ̂[row_index, col_index]; # assign value to Σ̂ₚ
        end
    end

    (ḡₚ, Σ̂ₚ); # return
end;

Now let's build the simulated annealing model instance using [the `build(...)` method](src/Factory.jl). We [initialize a `model::MySimulatedAnnealingMinimumVariancePortfolioAllocationProblem` instance](src/Types.jl) with initial weights `w`, the portfolio-specific expected return vector `ḡₚ`, covariance matrix `Σ̂ₚ`, and target return `R = 0.15` (15% annualized return):

In [12]:
model = build(MySimulatedAnnealingMinimumVariancePortfolioAllocationProblem, (
    w = 0.01*ones(length(my_list_of_tickers)),
    ḡ = ḡₚ,
    Σ̂ = Σ̂ₚ,
    R = desired_target_return, # desired return
));

Now let's solve the minimum variance portfolio allocation problem using the simulated annealing algorithm. 

We [call the `solve(...)` method](src/Compute.jl) with the model instance and algorithm parameters including temperature schedule, cooling rate, and penalty parameters. 

The method returns an updated model with optimized weights, which we extract and store in the `ŵ::Array{Float64,1}` variable:

In [13]:
ŵ = let
    
    # initialize -
    K = 200*length(my_list_of_tickers);
    Tₒ = 1.0 # initial T

    # TODO: Tune these parameters as needed (must pass the two assert statements below)
    T₁ = 0.1; # final T 
    α = 0.10; # cooling rate
    β = 0.01; # parameter perturbation
    τ = 0.90; # rate of decrease of the penalty/barrier parameters
    μ = 0.5; # penalty/barrier parameter for non-negativity
    ρ = 0.1; # penalty/barrier parameter for return and sum-to-one constraints

    # TODO: Implement the simulated annealing algorithm to solve the minimum variance portfolio allocation problem
    # TODO: SA implementation goes the src/Compute.jl file in the "solve" function
    # TODO: Once SA has been implemented, uncomment the following lines will call the solver
    # updated_model = solve(model, K = K, T₀ = Tₒ, T₁ = T₁,
    #     α = α, β = β, τ = τ, μ = μ, ρ = ρ);    

    updated_model.w; # return the updated weights
end;

UndefVarError: UndefVarError: `updated_model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Let's check the constraints. The sum of the weights should be 1, and the expected return should be close to the target return.

__Check sum constraint:__ We'll use the [`isapprox(...)` function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to check if the sum of the weights is approximately equal to 1.0, within an absolute tolerance of `atol` = 1e-2.

In [14]:
@assert isapprox(sum(ŵ), 1.0, atol=1e-2)

UndefVarError: UndefVarError: `ŵ` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

__Check expected return constraint:__ We'll use the [`isapprox(...)` function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to check if the expected return of the portfolio is __at least equal__ (approximately) to the target return `R`, within an absolute tolerance of `atol` = 1e-2.

> __What do we do if the constraints are not satisfied?__ 
>
> If the return or sum constraints are not satisfied, we can adjust the penalty parameters `μ` and `ρ` to increase the strength of the penalty terms in the objective function. This will encourage the algorithm to find a solution that satisfies the constraints more closely. We can also consider increasing the number of iterations or adjusting the cooling schedule to allow the algorithm more time to explore the solution space.
>
> __Required:__ Play with the hyperparameters until you pass these checks.

So what happened?

In [15]:
r = transpose(ŵ)*ḡₚ;
println("Portfolio expected return: ", r)
@assert isapprox(transpose(ŵ)*ḡₚ, desired_target_return, atol=1e-2)

UndefVarError: UndefVarError: `ŵ` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

What is the portfolio risk using the weights computed using Simulated Annealing?

In [16]:
let
    risk = transpose(ŵ)*Σ̂ₚ*ŵ # portfolio variance
    println("Risk using Simulated Annealing: $(risk |> x-> round(x, digits=4))");
end

UndefVarError: UndefVarError: `ŵ` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

## Task 3: Let's check our results using another method
Traditionally, the minimum variance portfolio allocation problem is solved using quadratic programming techniques. Let's compare our results from the Simulated Annealing algorithm to those obtained using [the `MadNLP.jl` package](https://github.com/MadNLP/MadNLP.jl).

In [17]:
true_model = let

    # initialize -
    number_of_firms = length(my_list_of_tickers); # how many firms do we have
    wₒ = zeros(number_of_firms) # initial allocation (zeros)
    wₒ[1] = 1.0; # all money in firm 1
    bounds = zeros(number_of_firms,2); 
    bounds[:,2] .= 1.0;

    # build the true model -
    true_model = build(MyMarkowitzRiskyAssetOnlyPortfolioChoiceProblem, (
        μ = ḡₚ,
        Σ = Σ̂ₚ,
        initial = wₒ,
        bounds = bounds,
        R = desired_target_return,
    ));
    
    true_model; # return
end;

 We pass the updated `true_model` object to the [solve(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/portfolio/#VLQuantitativeFinancePackage.solve-Tuple{MyMarkowitzRiskyAssetOnlyPortfolioChoiceProblem}) (which initializes and solves the optimization problem). The solution returned from the [solve(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/portfolio/#VLQuantitativeFinancePackage.solve-Tuple{MyMarkowitzRiskyAssetOnlyPortfolioChoiceProblem}) is stored in the `solution` dictionary. 

> __Status__: We check the `status` of the solution. If the `status` indicates an optimal solution was found, we store that solution in the `efficient_frontier` dictionary. In particular, we store the portfolio risk as the `key` and the expected return, risk, and allocation as the `value` in the `efficient_frontier` dictionary.

Why the [try-catch environment](https://docs.julialang.org/en/v1/base/base/#try)? The [solve(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/portfolio/#VLQuantitativeFinancePackage.solve-Tuple{MyMarkowitzRiskyAssetOnlyPortfolioChoiceProblem}) has an [@assert statement](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check if the calculation has converged. Thus, the solve method can [throw](https://docs.julialang.org/en/v1/base/base/#Core.throw) an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) if the optimization problem fails to converge. To gracefully handle this case, we use a [try-catch construct](https://docs.julialang.org/en/v1/base/base/#try). See the [is_solved_and_feasible method from the JuMP package](https://jump.dev/JuMP.jl/stable/api/JuMP/#JuMP.is_solved_and_feasible) for more information.

Let's save the risk, reward, and allocation from the `MadNLP.jl` solution in the `risk_madnlp::Float64`, `reward_madnlp::Float64`, and `allocation_madnlp::Array{Float64,1}` variables, respectively.

In [18]:
(risk_madnlp, reward_madnlp, allocation_madnlp) = let

    # initialize -
    risk_value = nothing;
    reward_value = nothing;
    allocation = nothing;

    try
        solution = solve(true_model); # solve the problem
        status_flag = solution["status"];    

        if (status_flag == MathOptInterface.LOCALLY_SOLVED)
            risk_value = solution["objective_value"]; # get the sqrt of the risk -
            reward_value = solution["reward"];
            allocation = solution["argmax"]; # TODO: need to update this key        
        end
    catch err
        # Uncomment to see which R's failed ...
        println("Failed: desired R = $(true_model.R). Infeasible");
    end    

    (risk_value, reward_value, allocation); # return
end;

What is the risk for the portfolio using the weights computed using [the `MadNLP.jl` package](https://github.com/MadNLP/MadNLP.jl)?

In [19]:
println("Risk using MadNLP: $(risk_madnlp |> x-> round(x, digits=4))");

Risk using MadNLP: 0.0169


__What's in each portfolio?__ Let's make a table using [the `pretty_table(...)` method exported from the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl) and compare the allocations from both methods.

In [20]:
let

    # initialize -
    df = DataFrame();

    for i ∈ eachindex(my_list_of_tickers)
        ticker = my_list_of_tickers[i];
        madnlp_alloc = allocation_madnlp[i] |> x-> round(x, digits=4);
        sa_alloc = ŵ[i] |> x-> round(x, digits=4);
        push!(df, (Ticker = ticker, MadNLP = madnlp_alloc, SimulatedAnnealing = sa_alloc));
    end

    # add a total row -
    total_madnlp = sum(allocation_madnlp);
    total_sa = sum(ŵ);
    push!(df, (Ticker = "Total", MadNLP = total_madnlp, SimulatedAnnealing = total_sa));

    # make a table -
    pretty_table(df, backend = :text, fit_table_in_display_vertically = false,
         table_format = TextTableFormat(borders = text_table_borders__compact));  

end

UndefVarError: UndefVarError: `ŵ` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

### Hmm. Interesting. Let's play around with our SA implementation
When we compare the allocations from both methods, both methods produce allocation vectors that sum (approximately) to 1 and have somewhat similar (not identical) risk and return characteristics. However, the individual weights are different!

This is not surprising, as the Simulated Annealing algorithm is a heuristic method that does not guarantee finding the global optimum. Where are the differences?

__Differences to consider:__
* __Small allocations:__ One obvious difference is that the simulated annealing method with the non-negativity barrier terms avoids allocating small fractions to any asset, while [the `MadNLP.jl` solution](https://github.com/MadNLP/MadNLP.jl) has no problem doing so. This is because the barrier term in the objective function penalizes small weights heavily, pushing them away from zero. 
* __Hmm:__ Could we get rid of the barrier term in the objective function and just use a penalty term for the non-negativity constraint? Suppose when we generate a new candidate solution, we set any negative weights to zero: $w^{\prime}_i \gets \max(0, w^{\prime}_i)$, where $w^{\prime} \gets w_{c} + \beta\cdot\texttt{randn}(\texttt{size}(w_{c}))$ and $w_{c}$ is the current solution. If we did this, we would enforce the non-negativity constraint directly, rather than through a barrier term in the objective function, and our objective function would be:
$$
\boxed{
\begin{align*}
    \min_{\mathbf{w}\in\mathbb{R}^M}\;P_{\mu,\rho}(\mathbf{w})\;&=\;\mathbf{w}^{\top}\mathbf{\Sigma}_{g}\mathbf{w}\;+\;\frac{1}{2\rho}\Bigr[\bigl(\mathbf{w}^\top ḡ - R^{*}\bigr)^2\;+\;\bigl(1-\sum_{i=1}^M w_i\bigr)^2\Bigl]\quad\blacksquare\\
\end{align*}}
$$

Thus, we could get rid of the barrier term in the objective function. Implement these changes and describe what happens to the results (assume the same hyperparameters as before).

In [21]:
# Put the analysis of your results here.

In [22]:
did_I_answer_the_discussion_question_1 = nothing; # Required: set to {true | false} if you have (or have not) answered the discussion question 1 above.

## Tests
The code block below shows how we implemented the tests and what we are testing. In these tests, we check values in your notebook and give feedback on which items are correct, missing, etc.

In [23]:
@testset verbose = true "CHEME 5800 PS4 Test Suite" begin

    @testset "Task 1: Compute the growth rate matrix" begin
        @testset "Growth rate array computation" begin
            @test isdefined(Main, :growth_rate_array) == true
            @test size(growth_rate_array, 1) > 0  # Has rows (trading days)
            @test size(growth_rate_array, 2) == length(list_of_tickers)  # Columns match number of tickers
        end

        @testset "Expected growth rate vector" begin
            @test isdefined(Main, :ḡ) == true
            @test length(ḡ) == length(list_of_tickers)
            @test all(isfinite.(ḡ))  # All values are finite
        end

        @testset "Covariance matrix" begin
            @test isdefined(Main, :Σ̂) == true
            @test size(Σ̂) == (length(list_of_tickers), length(list_of_tickers))
            @test all(isfinite.(Σ̂))  # All values are finite
            @test issymmetric(Σ̂)  # Covariance matrix should be symmetric
        end
    end

    @testset "Task 2: Solve the optimal allocation problem using Simulated Annealing" begin
        @testset "Portfolio ticker selection" begin
            @test isdefined(Main, :my_list_of_tickers) == true
            @test length(my_list_of_tickers) > 0
            @test all(ticker -> ticker in list_of_tickers, my_list_of_tickers)
        end

        @testset "Portfolio parameters computation" begin
            @test isdefined(Main, :ḡₚ) == true
            @test isdefined(Main, :Σ̂ₚ) == true
            @test length(ḡₚ) == length(my_list_of_tickers)
            @test size(Σ̂ₚ) == (length(my_list_of_tickers), length(my_list_of_tickers))
            @test all(isfinite.(ḡₚ))
            @test all(isfinite.(Σ̂ₚ))
        end

        @testset "Simulated annealing model" begin
            @test isdefined(Main, :model) == true
            @test typeof(model) == MySimulatedAnnealingMinimumVariancePortfolioAllocationProblem
            @test length(model.w) == length(my_list_of_tickers)
            @test model.R == desired_target_return
        end

        @testset "Optimization results" begin
            @test isdefined(Main, :ŵ) == true
            @test length(ŵ) == length(my_list_of_tickers)
            @test all(w -> w >= 0, ŵ)  # Non-negativity constraint
            @test isapprox(sum(ŵ), 1.0, atol=1e-2)  # Sum-to-one constraint
            @test isapprox(transpose(ŵ)*ḡₚ, model.R, atol=1e-2)  # Target return constraint
        end

        @testset "Portfolio risk calculation" begin
            risk_sa = transpose(ŵ)*Σ̂ₚ*ŵ
            @test isfinite(risk_sa)
            @test risk_sa > 0  # Risk should be positive
        end
    end

    @testset "Task 3: Check results using another method" begin
        @testset "MadNLP model construction" begin
            @test isdefined(Main, :true_model) == true
            @test typeof(true_model) == MyMarkowitzRiskyAssetOnlyPortfolioChoiceProblem
            @test true_model.R == desired_target_return
        end

        @testset "MadNLP solution" begin
            @test isdefined(Main, :risk_madnlp) == true
            @test isdefined(Main, :reward_madnlp) == true
            @test isdefined(Main, :allocation_madnlp) == true
            
            # Check if solution was obtained (not nothing)
            @test risk_madnlp !== nothing
            @test reward_madnlp !== nothing
            @test allocation_madnlp !== nothing
            
            # If solution exists, check properties
            if risk_madnlp !== nothing
                @test isfinite(risk_madnlp)
                @test risk_madnlp > 0
                @test isapprox(reward_madnlp, desired_target_return, atol=1e-2)
                @test length(allocation_madnlp) == length(my_list_of_tickers)
                @test all(w -> w >= 0, allocation_madnlp)
                @test isapprox(sum(allocation_madnlp), 1.0, atol=1e-4)
            end
        end

        @testset "Comparative analysis" begin
            # Check that both methods produced valid allocations
            @test length(allocation_madnlp) == length(ŵ)
            
            # Both should satisfy basic constraints
            @test isapprox(sum(ŵ), 1.0, atol=1e-2)
            if allocation_madnlp !== nothing
                @test isapprox(sum(allocation_madnlp), 1.0, atol=1e-4)
            end
            @test did_I_answer_the_discussion_question_1 == true;
        end
    end
end;

Optimization results: Test Failed at Test Failed at /Users/jeffreyvarner/Desktop/julia_work/CHEME-5800-Fall-2025/PS4-CHEME-5800-TEMPLATE-F2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y100sZmlsZQ==.jl:48
  Expression: isdefined(Main, :ŵ) == true
   Evaluated: false == true

Stacktrace:
/Users/jeffreyvarner/Desktop/julia_work/CHEME-5800-Fall-2025/PS4-CHEME-5800-TEMPLATE-F2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y100sZmlsZQ==.jl:48
  Expression: isdefined(Main, :ŵ) == true
   Evaluated: false == true

Stacktrace:
 [1] top-level scope
   @ ~/Desktop/julia_work/CHEME-5800-Fall-2025/PS4-CHEME-5800-TEMPLATE-F2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y100sZmlsZQ==.jl:3
 [2] macro expansion
   @ ~/.julia/juliaup/julia-1.12.1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.12/Test/src/Test.jl:1776 [inlined]
 [3] macro expansion
   @ ~/Desktop/julia_work/CHEME-5800-Fall-2025/PS4-CHEME-5800-TEMPLATE-F2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y100sZm

Test.TestSetException: Some tests did not pass: 39 passed, 2 failed, 7 errored, 0 broken.

___

## Summary
In this activity, we implemented the simulated annealing algorithm to solve the minimum variance portfolio allocation problem, a constrained optimization challenge in quantitative finance. We formulated the problem with penalty and barrier terms to handle constraints, trained the model on historical S&P 500 data, and validated our results against traditional quadratic programming methods.

> __Key Takeaways:__
> 
> * **Augmented objective function with constraints:** We incorporated inequality constraints (non-negativity) using logarithmic barrier terms and equality constraints (target return and sum-to-one) using quadratic penalty terms. The penalty parameters $\mu$ and $\rho$ decreased during optimization to enforce constraints more strictly.
> * **Simulated annealing parameter tuning:** We configured the algorithm with an initial temperature of 1.0, and an adaptive iteration count based on portfolio size. You chose the hyperparameters so that the constraints were satisfied, and the desired return was achieved.
> * **Validation through comparative analysis:** We compared simulated annealing results against MadNLP quadratic programming solutions, observing similar portfolio risk and return characteristics but different individual asset allocations. The barrier term approach in simulated annealing naturally avoided small allocations, producing sparser portfolio weights.

The simulated annealing algorithm successfully found feasible portfolio allocations satisfying all constraints, demonstrating the effectiveness of heuristic methods for solving constrained optimization problems in financial applications.
___